In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Asus F15\\Text-summarization\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Asus F15\\Text-summarization'

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: Path
    ALL_REQUIRED_FILES: list
    

In [26]:
from textsummarizer.constants import *
from textsummarizer.utils.common import read_yaml, create_directories

In [27]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath= CONFIG_FILE_PATH,
        params_filepath= PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
        root_dir = config.root_dir,
        STATUS_FILE = config.STATUS_FILE,
        ALL_REQUIRED_FILES = config.ALL_REQUIRED_FILES,
        )
        return data_validation_config


In [28]:
import os 
from textsummarizer.logging import logger

In [53]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_files_exist(self) -> bool:
        try:
            # List all files/folders in the extracted dataset
            dataset_path = os.path.join("artifacts", "data_ingestion", "samsum_dataset")
            all_files = os.listdir(dataset_path)

            # Check for missing required files/folders
            missing_files = [f for f in self.config.ALL_REQUIRED_FILES if f not in all_files]
            validation_status = len(missing_files) == 0

            # Write status to file
            with open(self.config.STATUS_FILE, 'w') as f:
                f.write(f"Validation status: {validation_status}")
                if missing_files:
                    f.write(f"\nMissing files/folders: {missing_files}")

            return validation_status

        except Exception as e:
            raise e


In [54]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_files_exist()
except Exception as e:
    raise e

[2025-08-17 17:46:04,535: INFO: Yaml file: config\config.yaml loaded successfully.]
[2025-08-17 17:46:04,541: INFO: Yaml file: params.yaml loaded successfully.]
[2025-08-17 17:46:04,545: INFO: created directory at: artifacts]
[2025-08-17 17:46:04,551: INFO: created directory at: artifacts/data_validation]
